# youtube 크롤링 코드

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# 1. 크롬 옵션 (창 없이 실행 가능)
options = Options()
options.add_argument('--headless')  # 창 안 띄우고 실행하려면 주석 해제
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 2. 드라이버 경로
driver_path = 'chromedriver.exe'
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 3. 유튜브 채널 URL 접속
channel_url = "https://www.youtube.com/@%EB%8C%95%EC%AA%BD%EC%9D%B4%EC%83%81%EB%8B%B4%EC%86%8C/videos"
driver.get(channel_url)
time.sleep(3)

# 4. 스크롤 내려서 더 많은 영상 로드 (선택)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")

for _ in range(5):  # 5번 정도 스크롤 내려서 영상 추가 로드
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 5. a 태그에서 video-title-link 수집
video_elements = driver.find_elements(By.ID, "video-title-link")
print(f"✅ 영상 수: {len(video_elements)}")

video_urls = []

for elem in video_elements:
    href = elem.get_attribute("href")
    title = elem.text.strip()
    if href:
        video_urls.append(href)
        print(f"🎬 {title}")
        print(f"🔗 {href}")

# 6. 저장
with open("all_video_urls.txt", "w", encoding="utf-8") as f:
    for url in video_urls:
        f.write(url + "\n")

print("\n✅ 수집 완료! 'all_video_urls.txt'에 저장됨.")

driver.quit()


✅ 영상 수: 67
🎬 “이사 후 분리불안, 2주간 출근도 못한 보호자의 이야기” | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=xEPaTtcysoM&pp=0gcJCYQJAYcqIYzv
🎬 “외출이 불가능해요…” 분리불안에 사람까지 무는 파양견 이야기 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=5UJ9dULd2Tc
🎬 [고민상담소]”잠도 못자고 너무 힘들어요..”몸이 불편한 구독자님의 절박한 요청에 직접 해결하러 나섰습니다
🔗 https://www.youtube.com/watch?v=MDLfhtfHEXw
🎬 퇴근 후 오랜만에 삼겹살~🥓의 하이라이트(0:00 ~ 4:40)
🔗 https://www.youtube.com/watch?v=LTCNHuiK_vQ
🎬 "엄마랑 산책 안 간다고!!" 끝까지 버티는 “줄다리기왕” 36kg 풍산 믹스 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=2wE460rf2Bk
🎬 산책 시 다른 강아지만 보면 폭주하는 “폭주왕” 페키니즈 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=cRKgJBNfMMY
🎬 유기견 보호소에서 안락사 직전 입양 됐지만 사람이 무서워 변까지 지리는 “찌질왕” 믹스견 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=qD9GqhVFITU
🎬 사람 아이처럼 키워져서 결국 독립을 못한 "불안왕" 푸들 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=4rgd2m3O1MU&pp=0gcJCYQJAYcqIYzv
🎬 짖음으로 인한 층간소음으로 민원을 받게 만든 “고음왕” 셔틀랜드 쉽독 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=gk3w9z2

In [6]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from youtube_transcript_api import YouTubeTranscriptApi

# 1. 크롬 옵션
options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver_path = 'chromedriver.exe'
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)

video_data = []  # 결과 저장할 리스트

try:
    # 2. 유튜브 채널 접속
    channel_url = "https://www.youtube.com/@%EB%8C%95%EC%AA%BD%EC%9D%B4%EC%83%81%EB%8B%B4%EC%86%8C/videos"
    driver.get(channel_url)
    time.sleep(3)

    # 3. 스크롤 내려서 영상 로딩
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 4. 영상 링크 수집
    video_elements = driver.find_elements(By.ID, "video-title-link")
    print(f"✅ 영상 수: {len(video_elements)}")

    video_urls = []
    for elem in video_elements:
        href = elem.get_attribute("href")
        title = elem.text.strip()
        if href:
            video_urls.append((href, title))

finally:
    driver.quit()

# 5. 자막 크롤링
print("\n🎬 자막 크롤링 시작!")

for idx, (url, title) in enumerate(video_urls):
    try:
        if "watch?v=" in url:
            video_id = url.split("watch?v=")[1]
        else:
            print(f"❌ URL 형식 오류: {url}")
            continue

        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
        full_text = ' '.join([entry['text'] for entry in transcript])

        video_data.append({
            'id': idx + 1,
            'video_id': video_id,
            'video_url': url,
            'title': title,
            'caption': full_text
        })

        print(f"✅ [{idx+1}] {title} 자막 수집 성공")

    except Exception as e:
        print(f"❌ [{idx+1}] {title} 자막 수집 실패: {e}")

# 6. CSV로 저장
csv_file = "youtube_captions.csv"

with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['id', 'video_id', 'video_url', 'title', 'caption'])
    writer.writeheader()
    for data in video_data:
        writer.writerow(data)

print(f"\n✅ 모든 데이터가 '{csv_file}'로 저장 완료!")


✅ 영상 수: 67

🎬 자막 크롤링 시작!
❌ [1] “이사 후 분리불안, 2주간 출근도 못한 보호자의 이야기” | 강아지 문제 행동교정 | 댕쪽이상담소 자막 수집 실패: no element found: line 1, column 0
✅ [2] “외출이 불가능해요…” 분리불안에 사람까지 무는 파양견 이야기 | 강아지 문제 행동교정 | 댕쪽이상담소 자막 수집 성공
✅ [3] [고민상담소]”잠도 못자고 너무 힘들어요..”몸이 불편한 구독자님의 절박한 요청에 직접 해결하러 나섰습니다 자막 수집 성공
❌ [4] 퇴근 후 오랜만에 삼겹살~🥓의 하이라이트(0:00 ~ 4:40) 자막 수집 실패: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=LTCNHuiK_vQ! This is most likely caused by:

The video is unplayable for the following reason: Join this channel to get access to members-only content like this video, and other exclusive perks.

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already descri

In [7]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from youtube_transcript_api import YouTubeTranscriptApi

# 1. 크롬 옵션
options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver_path = 'chromedriver.exe'
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)

video_data = []  # 결과 저장할 리스트

try:
    # 2. 유튜브 채널 접속
    channel_url = "https://www.youtube.com/@%EA%B2%AC%EC%9D%B8%EC%82%AC%EC%9D%B4/videos"
    driver.get(channel_url)
    time.sleep(3)

    # 3. 스크롤 내려서 영상 로딩
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 4. 영상 링크 수집
    video_elements = driver.find_elements(By.ID, "video-title-link")
    print(f"✅ 영상 수: {len(video_elements)}")

    video_urls = []
    for elem in video_elements:
        href = elem.get_attribute("href")
        title = elem.text.strip()
        if href:
            video_urls.append((href, title))

finally:
    driver.quit()

# 5. 자막 크롤링
print("\n🎬 자막 크롤링 시작!")

for idx, (url, title) in enumerate(video_urls):
    try:
        if "watch?v=" in url:
            video_id = url.split("watch?v=")[1]
        else:
            print(f"❌ URL 형식 오류: {url}")
            continue

        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
        full_text = ' '.join([entry['text'] for entry in transcript])

        video_data.append({
            'id': idx + 1,
            'video_id': video_id,
            'video_url': url,
            'title': title,
            'caption': full_text
        })

        print(f"✅ [{idx+1}] {title} 자막 수집 성공")

    except Exception as e:
        print(f"❌ [{idx+1}] {title} 자막 수집 실패: {e}")

# 6. CSV로 저장
csv_file = "youtube_captions_soo.csv"

with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['id', 'video_id', 'video_url', 'title', 'caption'])
    writer.writeheader()
    for data in video_data:
        writer.writerow(data)

print(f"\n✅ 모든 데이터가 '{csv_file}'로 저장 완료!")


✅ 영상 수: 17

🎬 자막 크롤링 시작!
✅ [1] 강아지가 꼬리를 쫙 펴고 흔들지 않을 때 자막 수집 성공
✅ [2] 우리 강아지 앉아-기다려 교육하기 자막 수집 성공
✅ [3] 강아지끼리 서로 핥아주는 이유는? 자막 수집 성공
✅ [4] 강아지의 놀이 포즈, 플레이 바우 자막 수집 성공
✅ [5] '이리와!' 하면 오는 매너있는 강아지로 양육해요 자막 수집 성공
✅ [6] 왜 강아지는 산책하면서 자꾸 소변을 보나요? 자막 수집 성공
✅ [7] 우리 강아지가 침대에 오줌을 싸서 고민이에요 자막 수집 성공
✅ [8] 우리 강아지가 편안한 '엎드리기' 교육하기 자막 수집 성공
✅ [9] 우리 강아지 꼬리를 긴장감 없이 좌우로 흔들어요 #기분이좋아요 편 자막 수집 성공
✅ [10] 우리 강아지 꼬리가 축 처져 있어요 자막 수집 성공
✅ [11] 우리 강아지와 '눈 마주치기'로 소통해요 자막 수집 성공
✅ [12] 우리 강아지가 눈을 깜빡이면서 나를 쳐다봐요 #나를쳐다봄편 자막 수집 성공
✅ [13] 우리 강아지가 다른 강아지에게 민망하게 왜 이럴까요? #마운팅 편 자막 수집 성공
✅ [14] 우리 강아지 왜 앉아서 한 발을 들까요? #앉아서한발들기 편 자막 수집 성공
✅ [15] 우리 강아지는 원하는 때와 장소에서 '앉아'를 할 수 있나요? 자막 수집 성공
✅ [16] 우리 강아지 야단칠 땐 하품을 해요 #하품 편 자막 수집 성공
✅ [17] 우리 강아지 왜 몸을 털까요? #몸털기 편 자막 수집 성공

✅ 모든 데이터가 'youtube_captions_soo.csv'로 저장 완료!
